# Import Packages

# Upload files from cluster

# Convergence Checks

Covergence Tests:
   - First check was "Status" and "message" which seemed to look okay 
   - Then checked "nit" (number of iterations the fit was performing) and saw this was mostly 0 - not fitting!
   - We plotted a single input spectra, its inital params and the fit to see that the fit and initial param spectra were basically identical
   - So originally, it looked like good fits as we initialised our values with a spectral shape 
   - Checked how errors change (made make-shift errors to ensure relatively sensible errors)
   - How many steps taken whilst fitting the initial params to the data - test whether the fit was moving
   - Created histogram of the differences between initial parameters compared to fitted - to check the fits were moving
   - Plotting fit vs initial params to check gradient is not zero (same as checking jacobians in results !!)
    

In [ ]:
fit_results_1D = []
for fit_result in fit_results:
    fit_results_1D.append(fit_result.x)
    
fit_results_1D = np.concatenate(fit_results_1D)
    
all_columns_initial_params_1D = np.concatenate(all_columns_initial_params)

mplplot.scatter(fit_results_1D,all_columns_initial_params_1D)
mplplot.xlabel("Fit Results")
mplplot.ylabel("Initial Parameters")
mplplot.title("Fit Vs. Initial Parameters Plot ")

# Testing fitted Spectra against original input

LONG OUTPUTS

In [ ]:
with apfits.open(file_name) as test:
    data = test["SCI"].data.T
    display(trimmed_dfxLowToHigh.index)
#     print(trimmed_dfx1stLowToHigh.loc[11,"Direct_Y"])
    for source in range(len(trimmed_dfxLowToHigh['Direct_X'])):
        test_trace_input_data = np.sum(data[int(trimmed_dfxLowToHigh.loc[source,"Direct_X"])-2 : int(trimmed_dfxLowToHigh.loc[source,"Direct_X"])+2],axis=0)
        mplplot.figure(source)
        mplplot.plot(test_trace_input_data, label="Data")
        mplplot.plot(result[source],label='Fit')
        mplplot.plot(data[int(trimmed_dfxLowToHigh.Direct_X[source])],label='Init Params')
        mplplot.xlabel("Column Number")
        mplplot.ylabel("ADU (e$^{-1}$)")
        mplplot.title(f"Trace Number {source}")
        mplplot.legend()
        if test_trace_input_data.max() > 20:
            mplplot.ylim(0,1.5)
# See slight differences from data to fits as fit is from 25 rows where as data is sum of 4 rows across all columns
# Sharp peak at ~1400 unknown why this is happening (could be simulated CR hit)
# Added original errors and see more sudden unexpected peaks but fit still moved from initial params

# WHAT IS THIS DOING AGAIN?

In [2]:
statuses = [result.status for result in fit_results]
cov = [result.hess_inv for result in fit_results]
# print(np.array(cov).flatten().shape)
# print(*statuses, sep='\n')
mplplot.plot(statuses)

NameError: name 'fit_results' is not defined

# How many steps taken whilst fitting the initial params to the data

In [ ]:
steps_taken = [result.nit for result in fit_results]
objective_func_value = [result.jac for result in fit_results]
# print(objective_func_value)
# The more traces there are, the more iterations required to fit each column
mplplot.plot(steps_taken)


In [ ]:
# List of Differences between intial parameters set for fit to the optimised results returned
init_to_opt = []
for (param, init) in zip(fit_results, all_columns_initial_params):
    for (sing_param, sing_init) in zip(param.x, init):
        init_to_opt.append(sing_param - sing_init)
mplplot.hist(init_to_opt, bins = np.linspace(0,6,24))
# Zero differences for columns with 0 traces in them

# Plotting the traces???

In [ ]:
with apfits.open(file_name) as test:
    mplplot.figure(figsize=(10,10))
    mplplot.imshow(result[trimmed_dfxLowToHigh.sort_values("Direct_Y").index], aspect="auto")
    figure = mplplot.figure(figsize=(10, 10))
    ax = figure.add_subplot(1, 1, 1, projection=apwcs.WCS(test["SCI"].header))    
    ax.imshow(
        # SCI is the images from the simulation
        test["SCI"].data,
        norm=apvis.ImageNormalize(
            stretch=apvis.HistEqStretch(data=test["SCI"].data),
            interval=apvis.ZScaleInterval(),
        ),
        cmap="Greys",
    )
    # Set image limits
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    ax.scatter(
        sourceCoords.ra,
        sourceCoords.dec,
        marker="x",
        c="r",
        transform=ax.get_transform("world"),
    )
    ax.set_title(os.path.basename(file_name))
        
#     directPixels = getSourcePixelsForImage(file_name, frameListDir)
    print(file_name, directPixels[24],directPixels[77] if len(directPixels) > 50 else "Short")
        
    # for loop to create trace boxes for each source within source list
    # HOW DO WE CHANGE FOR C DIRECTION?? NEED TO LOAD C DATA???
    # where is sourceId defined? Within file_name???
    for sourceId, directPixel in enumerate(directPixels):
        traceBox1st = compute1stOrderTraceBox(
            directPixel,
            filter_=filter_,
            module=os.path.basename(file_name).split("_")[3][-2].upper(),
            direction="R",
            ec="orange",
            #ec="orange",
            lw="2",
            fc="none",
        )
        
        traceBox2nd = compute2ndOrderTraceBox(
            directPixel,
            filter_=filter_
            module=os.path.basename(file_name).split("_")[3][-2].upper(),
            direction="R",
            ec="blue",
            #ec="orange",
            lw="2",
            fc="none",
        )
        
        ax.add_patch(traceBox1st)
#         ax.add_patch(traceBox2nd)
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

# NEED to adjust position of applying sensitivity model for each source 

In [ ]:
SensFile = apfits.open(f"GRISM_NIRCAM/V2/NIRCam.{filter_}.{direction}.{module}.1st.sensitivity.fits")
# print(repr(SensFile[1].header)) # Units check
SensTable = aptable.Table(SensFile[1].data)
SensData_WL = SensTable[0][:]
SensData_Sens = SensTable[1][:]
SensData_Er = SensTable[2][:]

print((SensData_WL.min(),SensData_WL.max()))

# Set up the Spectral Response Function Curve to perform Flux calibration (from ADU to Flux)
# fill_value = extrapolate required as the 2 wavelength ranges do not match
SRFCurve = sp.interpolate.interp1d(SensData_WL,SensData_Sens,kind='cubic',fill_value="extrapolate")

SRFCurves = {1: SRFCurve}

mplplot.plot(SensData_WL,SRFCurve(SensData_WL))


In [ ]:
SensFile = apfits.open(f"GRISM_NIRCAM/V2/NIRCam.{filter_}.{direction}.{module}.2nd.sensitivity.fits")
# print(repr(SensFile[1].header)) # Units check
SensTable2nd = aptable.Table(SensFile2nd[1].data)
SensData_WL2nd = SensTable2nd[0][:]
SensData_Sens2nd = SensTable2nd[1][:]
SensData_Er2nd = SensTable[2][:]

print((SensData_WL2nd.min(),SensData_WL2nd.max()))

# Set up the Spectral Response Function Curve to perform Flux calibration (from ADU to Flux)
# fill_value = extrapolate required as the 2 wavelength ranges do not match
SRFCurve2nd = sp.interpolate.interp1d(SensData_WL2nd,SensData_Sens2nd,kind='cubic',fill_value="extrapolate")

SRFCurves.update({2:SRFCurve2nd})

mplplot.plot(SensData_WL2nd,SRFCurve2nd(SensData_WL2nd))

In [ ]:
resultSensCurve = np.zeros_like(result)
trace_WLs_list = []
trace_x_pixels_indexes = []

firstPrinted = False
secondPrinted = False

for res_index, res in enumerate(result):
    
    order = trimmed_dfxLowToHigh.Order[res_index]
    trace = computeTraceWLforXpixels(
            (trimmed_dfxLowToHigh.Direct_X[res_index], trimmed_dfxLowToHigh.Direct_Y[res_index]),
            filter_=filter_
            module=os.path.basename(file_name).split("_")[3][-2].upper(),
            direction="R",
    )
    # Temporary check that second order wavelength step per pixel is smaller
    # remove once verified.
    if order == 1 and not firstPrinted:
        print(order)
        print(trace[1]-trace[0])
    elif order == 2 and not secondPrinted:
        print(order)
        print(trace[1]-trace[0])

#     print(trace[0])
    trace_x_pixels = trace[0][(trace[0]>=0) & (trace[0]<2048)].astype(int)
    trace_x_pixels_indexes.append(trace_x_pixels)
    
    trace_WLs = trace[1][(trace[0]>=0) & (trace[0]<2048)]
    trace_WLs_list.append(trace_WLs)
#     print(trace_WLs,trace_WLs.shape)
    
#     try:
#         print(trace_x_pixels.min(),trace_x_pixels.max())
#     except:
#         print(trace_x_pixels)
    
#     print(len(trace_x_pixels))
    
    resultSensCurve[res_index,trace_x_pixels] = SRFCurves[order](trace_WLs)
    
mplplot.figure(figsize=(10,10))
mplplot.imshow(resultSensCurve[trimmed_dfxLowToHigh.sort_values("Direct_Y").index], aspect="auto")

calibratedFlux = result/resultSensCurve


mplplot.figure(figsize=(10,10))
mplplot.imshow(calibratedFlux[trimmed_dfxLowToHigh.sort_values("Direct_Y").index], aspect="auto",
               norm=apvis.ImageNormalize(
                stretch=apvis.HistEqStretch(data=calibratedFlux[trimmed_dfxLowToHigh.sort_values("Direct_Y").index]),
                interval=apvis.ZScaleInterval(),
        ))
    

## NEED to adjust position of applying sensitivity model for each source 

In [ ]:
for position, sub_result in enumerate(calibratedFlux):
    mplplot.figure(figsize=(9,9))
    wavelengths = trace_WLs_list[position]
    wavelengths_mask = (wavelengths>=2.5) & (wavelengths<3.9)
    mplplot.plot(wavelengths[wavelengths_mask], sub_result[trace_x_pixels_indexes[position]][wavelengths_mask])
    
    mplplot.title(f"Trace Number {position}")
    mplplot.xlabel('$\lambda$ / $\mu$m')
    mplplot.ylabel('Flux / erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$' )